In [1]:
import sys
import os
import re
import copy
import numpy as np
import random
import itertools
from typing import List, Optional 
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# load the various FastSim libraries
sys.path.insert(0, '../MATHUSLA_FastSim/')
import DetectorSimulation.Detector as Detector
import DetectorSimulation.Particle as Particle
import DetectorSimulation.Vertex as Vertex
import DetectorSimulation.lorentz_transformation as lt
import DetectorSimulation.llp_gun as llp_gun
## depending on which fvs are used load a different specialized llp gun (llp_gun_new) (built on top of llp_gun)
import DetectorSimulation.llp_gun_new as lg
## NOTE: the various hadronic functions in llp_gun_new must be updated with the correct path of the LLP hadronic decay 
## 4-vector files depending on the type of analysis 

## These hadronic functions are almost the same but were separated into different functions for ease of use with the 
## different LLP analyses - but should be very easy to update them for any new analysis that need be done

# load helper function library for later
from Helpers.functions import *

In [2]:
sys.path.insert(0,'../FastSim_Additions/')
from Additions import initiate_detector

In [3]:
from scipy.interpolate import interp1d

In [4]:
file = '../SimulationData/RHN_Ue_LLPweight4vectorBmesonlist_mN_0.316228.csv'
mass = float(file.split('_')[-1][:-4])

In [5]:
length_file = '../SimulationData/RHNctauUe.dat'
mass_to_length = np.loadtxt(length_file, skiprows = 1)
interp_mass = interp1d(mass_to_length.T[0], mass_to_length.T[1])
length_U1 = float(interp_mass(mass))

In [24]:
mixing = np.logspace(-4,-5, 100) 

In [30]:
lengths = length_U1 / mixing

In [31]:
lengths

array([  648.04081911,   678.89777056,   711.22399897,   745.08946508,
         780.56746086,   817.73476811,   856.67182467,   897.4628985 ,
         940.19627001,   984.96442318,  1031.86424566,  1080.99723849,
        1132.46973577,  1186.39313475,  1242.88413696,  1302.06500077,
        1364.06380597,  1429.01473094,  1497.0583431 ,  1568.34190308,
        1643.01968343,  1721.2533025 ,  1803.21207424,  1889.0733746 ,
        1979.0230254 ,  2073.25569654,  2171.97532726,  2275.39556751,
        2383.74024035,  2497.24382635,  2616.15197104,  2740.72201655,
        2871.22355856,  3007.93902973,  3151.16431097,  3301.2093718 ,
        3458.39894116,  3623.07321019,  3795.5885685 ,  3976.31837547,
        4165.65376825,  4364.00450829,  4571.79986813,  4789.48956045,
        5017.54471135,  5256.45887994,  5506.74912652,  5768.9571316 ,
        6043.65036823,  6331.42333011,  6632.8988182 ,  6948.72928861,
        7279.59826462,  7626.221816  ,  7989.35010871,  8369.76902843,
      

In [21]:
detector_benchmark = initiate_detector()
phi_min, phi_max, theta_min, theta_max = get_detector_angles(detector_benchmark)

In [32]:
fv_path = join(os.getcwd(), file)

in_detector = np.zeros(len(lengths))
decay_in_detector = np.zeros(len(lengths))

for i,ctau in enumerate(lengths):
    vectors = read_vectors(fv_path, 100)
    for vec in vectors:
        four_p = vec[1:]
        llp_theta = get_theta(four_p)
        
        if (llp_theta < theta_max) and (llp_theta > theta_min):
            #print('Particle in MATHUSLA')
            in_detector[i] += 1
            detector_benchmark.clear_detector()
            
            rot_four_p = deal_with_phi(four_p, phi_min, phi_max)
            
            pack = get_weight(rot_four_p, mass, ctau, detector_benchmark)
            if pack is not None:
                print('Particle Decayed in MATHUSLA')
                decay_in_detector[i] += 1

In [11]:
in_detector

array([0., 5., 1., 2., 4., 0., 0., 1., 4., 3., 1., 2., 0., 3., 1., 2., 2.,
       3., 3., 2., 2., 2., 3., 0., 1., 5., 2., 3., 1., 1., 2., 3., 2., 1.,
       1., 2., 5., 0., 5., 3., 2., 2., 0., 2., 1., 1., 2., 2., 3., 3., 4.,
       3., 3., 3., 2., 2., 1., 2., 1., 1., 2., 2., 1., 2., 2., 4., 2., 2.,
       5., 1., 1., 2., 0., 2., 1., 2., 2., 1., 4., 0., 4., 3., 0., 3., 1.,
       2., 1., 0., 5., 4., 2., 2., 0., 1., 1., 1., 1., 3., 5., 3.])

In [12]:
decay_in_detector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])